Let's now work on the model to try and predict the damage level of each building. Let's start by importing the necessary libraries and the dataset containing the one-hot signatures created in the previous notebook.

In [1]:
import torch
import pandas as pd
import numpy as np


In [ ]:
dataframe = pd.read_csv('signature_dataframe.csv')
dataframe.head()